In [ ]:
%matplotlib widget
from bmcs_shell.api import WBTessellation4P, WBShellAnalysis, MATS2DElastic
import numpy as np

In [ ]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=1
n_x_plus=1
wbm = WBShellAnalysis(h=10, tmodel=tmodel)

In [ ]:
data = dict(gamma=0.33, a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=False)
wbm.geo.trait_set(**data)

In [ ]:
wbm.xdomain.mesh.direct_mesh = False
wbm.xdomain.mesh.subdivision = 2

In [ ]:
mesh = wbm.xdomain.mesh.mesh
mesh

In [ ]:
wbm.interact()

Point, facets
You need the average of facets normal for each point

In [ ]:
X_Id = wbm.xdomain.mesh.X_Id
X_Id

In [ ]:
wbm.xdomain.mesh.I_Fi

In [ ]:
np.arange(I_Fi.shape[0])[:, np.newaxis].shape

In [ ]:
F_indices = np.arange(I_Fi.shape[0])[:, np.newaxis]
np.hstack((I_Fi[:, :2], F_indices))

In [ ]:
I_Li[:, :2]

In [ ]:
I_Li

In [ ]:
X_Id[I_Li]

In [ ]:
# Unique lines
I_Li = np.vstack(((I_Fi[:, (0, 1)]), (I_Fi[:, (0, 2)]), (I_Fi[:, (1, 2)])))
I_Li = np.sort(I_Li, axis=1)
I_Li = np.unique(I_Li, axis=0)
I_Li

In [ ]:
I_Fi_sorted[:, :2] == I_Li

In [ ]:
a = np.where(np.sum((I_Fi_sorted[:, 1:] == I_Li[0])*1, axis=1) == 2)[0]
a

In [ ]:
np.sum((I_Fi_sorted[:, 1:] == I_Li[:, np.newaxis])*1, axis=2)

In [ ]:
arr = np.sum((I_Fi_sorted[:, 1:] == I_Li[:, np.newaxis])*1, axis=2)
arr = np.where(arr == 2)
np.vstack((arr[0], arr[1])).T

In [ ]:
I_Fi_sorted[:, :2]

In [ ]:
arr = np.sum((I_Fi_sorted[:, :2] == I_Li[:, np.newaxis])*1, axis=2)
arr = np.where(arr == 2)
np.vstack((arr[0], arr[1])).T

In [ ]:
np.sum((I_Fi_sorted[:, :2] == I_Li[:, np.newaxis])*1, axis=2)

In [ ]:
b = np.where(np.sum((I_Fi_sorted[:, :2] == I_Li[:, np.newaxis])*1, axis=2) == 2)
b

In [ ]:
I_Fi_sorted = np.sort(I_Fi, axis=1)
I_Fi_sorted

In [ ]:
I_Fi

In [ ]:
I_Fi = wbm.xdomain.mesh.I_Fi
F_indices = np.arange(I_Fi.shape[0])[:, np.newaxis]
I_Li = np.vstack((np.hstack((np.sort(I_Fi[:, :2]), F_indices)), np.hstack((np.sort(I_Fi[:, 1:]), F_indices))))
# I_Li = np.vstack((np.hstack((I_Fi[:, :2], F_indices)), np.hstack((I_Fi[:, 1:], F_indices))))
I_Li

In [ ]:
np.sort(I_Li, axis=0)

In [ ]:
wbm.xdomain.norm_F_normals

In [ ]:
xmodel = wbm.fe_domain[0].xmodel
xmodel.mesh.I_CDij

In [ ]:
_, idx_remap = xmodel.mesh.unique_node_map

In [ ]:
idx_remap[xmodel.mesh.I_CDij]

In [ ]:
idx_remap[xmodel.mesh.I_CDij[(0, -1),:,(0, -1),:].flatten()]

In [ ]:
xmodel

In [ ]:
wbm.interact()

In [ ]:
import numpy as np
alpha_range = np.linspace(0.5, 1.3, 10)
a_range = np.linspace(300,500,10)

In [ ]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
import numpy as np
U_grid = []
sig_grid = []
for alpha in alpha_range:
    U_list = []
    sig_list = []
    for a in a_range:
        wbm = WBShellAnalysis(F=-20000, h=10, tmodel=tmodel)
        data = dict(alpha=alpha, a=a, a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
                    n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
        wbm.wb_mesh.trait_set(**data)
        max_U = wbm.get_max_vals()
        
        U1 = wbm.hist.U_t[-1]
        eps1 = wbm.fe_domain[0].xmodel.map_U_to_field(U1)
        sig1, _ = wbm.fe_domain[0].tmodel.get_corr_pred(eps1,1)
        sig_x, sig_y, tau = sig1.T
        sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float_))
        sig_val, sig_dir = np.linalg.eig(sig_ab)
        sig_list.append([np.min(sig_val), np.max(sig_val)])

        print(alpha, a, max_U)
        U_list.append(max_U)
    U_grid.append(U_list)
    sig_grid.append(sig_list)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(alpha_range, a_range)
Z = np.array(U_grid)
surf = ax.plot_surface(X, Y, Z, 
                       color='green',
                       #linewidth=0, antialiased=False)
                    alpha=.0, linewidth=5,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = np.array(sig_grid)[:,:,1]
surf = ax.plot_surface(X, Y, Z,color='red', 
                       #cmap=cm.copper,
                    alpha=.3, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = -np.array(sig_grid)[:,:,0]
surf = ax.plot_surface(X, Y, Z,color='blue', 
                       #cmap=cm.copper,
                    alpha=.6, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
ax.set_xlabel(r'$\alpha~[\mathrm{rad}]$')
ax.set_ylabel(r'$a~[\mathrm{mm}]$')
ax.set_zlabel(r'max compression [MPa]')


In [ ]:
i_alpha, i_a = 0, -1
wbm = WBShellAnalysis(F=-1000, tmodel=tmodel)
print('alpha', alpha_range[i_alpha],'a', a_range[i_a] )
data = dict(alpha=alpha_range[i_alpha], 
            a=a_range[i_a], a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data)
wbm.wb_mesh.interact()

In [ ]:
15000 / 39000 